In [ ]:
#Pre-requisites folder creation
#pip install pandas numpy glob os string tkinter faker csv json glob

import os

# Get the current working directory
base_location = os.getcwd()

# Directory names to be created
directory_names = ['1.schema', '2.output', '3.master', '4.updated', '5.json', '6.sql']

# Create directories using the current location as base location
for dir_name in directory_names:
    dir_path = os.path.join(base_location, dir_name)
    os.makedirs(dir_path, exist_ok=True)
    print(f"Directory '{dir_name}' created at '{dir_path}'")

In [ ]:
##Generate Test Data Based on Schema
import pandas as pd
import numpy as np
import glob
import random
import string
import os
from tkinter import Tk, filedialog
from faker import Faker

def browse_directory(prompt,default_path):
    Tk().withdraw()  # We don't want a full GUI, so keep the root window from appearing
    directory = filedialog.askdirectory(title=prompt,initialdir=default_path)
    return directory

# Set the default path for ease of use
default_schema_directory = os.path.join(os.getcwd(), "1.schema")
default_output_directory = os.path.join(os.getcwd(), "2.output")

# Get user input for the directory containing the schema files
schema_directory = browse_directory("Select the directory containing the schema CSV files",default_schema_directory)

# Get user input for the directory containing the output files
output_directory = browse_directory("Select the directory to save the test data CSV files",default_output_directory)

faker = Faker()

# Function to generate random string of a given length
def random_string(length):
    return ''.join(random.choices(string.ascii_lowercase, k=length))

# Function to generate random integer with a specific number of digits
def random_integer(min_digits, max_digits):
    min_value = 10**(min_digits - 1)
    max_value = 10**max_digits - 1
    return np.random.randint(min_value, max_value + 1, dtype=np.int64)

# Function to generate unique IDs
def generate_unique_ids(num_rows):
    return list(range(1, num_rows + 1))

# Function to generate data based on schema
def generate_data(schema, num_rows):
    data = {}
    for index, row in schema.iterrows():
        col = row[0]
        dtype_info = row[1]
        is_primary_key = len(row) > 2 and row[2] == 'primary_key'
        dtype_parts = dtype_info.split(':')
        dtype = dtype_parts[0]
        min_length = int(dtype_parts[1]) if len(dtype_parts) > 1 else None
        max_length = int(dtype_parts[2]) if len(dtype_parts) > 2 else None
        permissible_values = dtype_parts[3].split('|') if len(dtype_parts) > 3 else None
        
        if is_primary_key:
            data[col] = generate_unique_ids(num_rows)
        elif permissible_values:
            data[col] = np.random.choice(permissible_values, num_rows)
        elif dtype == 'int':
            if min_length and max_length:
                data[col] = [random_integer(min_length, max_length) for _ in range(num_rows)]
            else:
                data[col] = np.random.randint(1, 100, num_rows)
        elif dtype == 'float':
            data[col] = np.random.rand(num_rows) * 100
        elif dtype == 'name':
            data[col] = [faker.name() for _ in range(num_rows)]
        elif dtype == 'date':
            data[col] = [faker.date() for _ in range(num_rows)]
        elif dtype == 'datetime':
            data[col] = [faker.date_time().isoformat() for _ in range(num_rows)]
        elif dtype == 'city':
            data[col] = [faker.city() for _ in range(num_rows)]
        elif dtype == 'country':
            data[col] = [faker.country() for _ in range(num_rows)]
        elif dtype == 'state':
            data[col] = [faker.state() for _ in range(num_rows)]
        elif dtype == 'zip':
            data[col] = [faker.zipcode() for _ in range(num_rows)]
        elif dtype == 'address_line1':
            data[col] = [faker.street_address() for _ in range(num_rows)]
        elif dtype == 'address_line2':
            data[col] = [faker.secondary_address() for _ in range(num_rows)]
        elif dtype == 'bool_1_0':
            data[col] = np.random.choice([1, 0], num_rows)
        elif dtype == 'str':
            if min_length and max_length:
                lengths = np.random.randint(min_length, max_length + 1, num_rows)
                data[col] = [random_string(length) for length in lengths]
            elif min_length:
                data[col] = [random_string(min_length) for _ in range(num_rows)]
            else:
                data[col] = np.random.choice(['A', 'B', 'C'], num_rows)
        else:
            data[col] = np.random.choice(['unknown'], num_rows)
    return pd.DataFrame(data)

# Load all transposed schema files
schema_files = glob.glob(os.path.join(schema_directory, 'schema_*.csv'))  # Adjust the pattern if needed
# Generate and save test data for each schema file
num_rows = 50
for schema_file in schema_files:
    schema = pd.read_csv(schema_file, header=None)
    test_data = generate_data(schema, num_rows)
    output_file = os.path.join(output_directory, os.path.basename(schema_file).replace('schema_', ''))  # Create corresponding output file name
    test_data.to_csv(output_file, index=False)
    print(f"Test data generated and saved to '{output_file}'")


In [ ]:
##Generate Master Data for Party, Account, Party Account Bridge
import pandas as pd
import numpy as np
from tkinter import Tk, filedialog

def browse_directory(prompt,default_path):
    Tk().withdraw()  # We don't want a full GUI, so keep the root window from appearing
    directory = filedialog.askdirectory(title=prompt,initialdir=default_path)
    return directory

# Set the default path for ease of use
default_master_directory = os.path.join(os.getcwd(), "3.master")

# Get user input for the directory to save the files
save_directory = browse_directory("Select the directory to save the master CSV files",default_master_directory)

# Function to generate unique 18-digit party numbers
def generate_unique_party_numbers(num_rows):
    unique_numbers = set()
    while len(unique_numbers) < num_rows:
        unique_numbers.add(str(np.random.randint(10**17, 10**18, dtype=np.int64)))
    return list(unique_numbers)

# Function to generate unique 10-digit account numbers
def generate_unique_account_numbers(num_rows):
    unique_numbers = set()
    while len(unique_numbers) < num_rows:
        unique_numbers.add(str(np.random.randint(10**9, 10**10, dtype=np.int64)))
    return list(unique_numbers)

# Function to define roles
def define_roles(num_rows):
    roles = ['Primary', 'Secondary']
    return np.random.choice(roles, num_rows)


# Number of rows for each file
num_rows_account = 50
num_rows_party = num_rows_account * 2

# Generate data
party_numbers = generate_unique_party_numbers(num_rows_party)
account_numbers = generate_unique_account_numbers(num_rows_account)
roles = define_roles(num_rows_account)

# Create DataFrames
df_party = pd.DataFrame({'party_num': party_numbers})
df_account = pd.DataFrame({'acct_num': account_numbers})
df_role = pd.DataFrame({'role': roles})




# Save DataFrames to CSV files
df_party.to_csv(os.path.join(save_directory,'master_party_numbers.csv'), index=False)
df_account.to_csv(os.path.join(save_directory,'master_account_numbers.csv'), index=False)
#df_role.to_csv(os.path.join(save_directory,'master_roles.csv'), index=False)

# Combine data into a single DataFrame ensuring no account, role combination have the same customer
combined_data = []
used_combinations = set()

for i in range(num_rows_account):
    account = account_numbers[i]
    role = roles[i]
    customer = party_numbers[i]
    if (account, role) not in used_combinations:
        combined_data.append({'Customer': customer, 'Account': account, 'Role': role})
        used_combinations.add((account, role))

# Add remaining party numbers to the combined data
remaining_customers = set(party_numbers) - set([entry['Customer'] for entry in combined_data])
for customer in remaining_customers:
    account = np.random.choice(account_numbers)
    role = np.random.choice(roles)
    combined_data.append({'Customer': customer, 'Account': account, 'Role': role})

# Ensure all account numbers are used at least once
used_accounts = set([entry['Account'] for entry in combined_data])
remaining_accounts = set(account_numbers) - used_accounts
for account in remaining_accounts:
    customer = np.random.choice(party_numbers)
    role = np.random.choice(roles)
    combined_data.append({'Customer': customer, 'Account': account, 'Role': role})

# Ensure all party numbers are used at least once
used_customers = set([entry['Customer'] for entry in combined_data])
remaining_customers = set(party_numbers) - used_customers
for customer in remaining_customers:
    account = np.random.choice(account_numbers)
    role = np.random.choice(roles)
    combined_data.append({'Customer': customer, 'Account': account, 'Role': role})

df_combined = pd.DataFrame(combined_data)

# Save the combined DataFrame to a CSV file
#df_combined.to_csv('master_party_account_role.csv', index=False)

print(f"Master data generated and saved to '{save_directory}'")





In [ ]:
##Replace Party, Account, Party Account with Master Data for joining on key columns

import pandas as pd
import os
import csv
import shutil
from tkinter import Tk, filedialog


# Function to browse and select multiple files
def browse_files(prompt,default_path):
    Tk().withdraw()  # We don't want a full GUI, so keep the root window from appearing
    file_paths = filedialog.askopenfilenames(title=prompt, initialdir=default_path, filetypes=[("CSV files", "*.csv")])
    return file_paths

# Set the default path for ease of use
default_output_directory = os.path.join(os.getcwd(), "2.output")
default_master_directory = os.path.join(os.getcwd(), "3.master")
default_updated_directory = os.path.join(os.getcwd(), "4.updated")


# Get user input for the master and target files
master_party_file = filedialog.askopenfilename(title="Select the master party CSV file",  filetypes=[("CSV files", "*.csv")], initialdir=default_master_directory)
master_account_file = filedialog.askopenfilename(title="Select the master account CSV file", filetypes=[("CSV files", "*.csv")],initialdir=default_master_directory)
target_files = browse_files("Select the target CSV files",default_output_directory)

# Load the master files
df_master_party = pd.read_csv(master_party_file, dtype={'party_num': str})
df_master_account = pd.read_csv(master_account_file, dtype={'acct_num': str})

# Convert column names to lowercase
df_master_party.columns = df_master_party.columns.str.lower()
df_master_account.columns = df_master_account.columns.str.lower()

# Function to update target files with referential integrity
def update_target_file(target_file, key_column, master_df):
    df_target = pd.read_csv(target_file, dtype={key_column: str})
    df_target.columns = df_target.columns.str.lower()
    df_unique_target = df_target.drop_duplicates(subset=[key_column])
    df_unique_master = master_df.drop_duplicates(subset=[key_column])

    if len(df_unique_target) <= len(df_unique_master):
        df_unique_target[key_column] = df_unique_master[key_column].values[:len(df_unique_target)]
    else:
        raise ValueError(f"Target count is greater than master count for {target_file}. Replacement not possible.")

    output_file = os.path.join(os.path.dirname(target_file), 'updated_' + os.path.basename(target_file))
    df_unique_target.to_csv(output_file, index=False)
    print(f"Column values replaced successfully! The updated file is saved as '{output_file}'")
    return df_unique_target
# Update each target file
for target_file in target_files:
    if 'party' in os.path.basename(target_file).lower() and 'acct' in os.path.basename(target_file).lower():
        # doing for party
        df_updated = update_target_file(target_file, 'party_num', df_master_party)
        output_file = os.path.join(os.path.dirname(target_file), 'updated_' + os.path.basename(target_file))
        df_updated.to_csv(output_file, index=False)
        print(f"Party column values replaced successfully! The intermediate file is saved as '{output_file}'")
        # doing for acct
        df_updated = update_target_file(output_file, 'acct_num', df_master_account)
        df_updated.to_csv(output_file, index=False)
        print(f"Account column values replaced successfully! The final updated file is saved as '{output_file}'")
    if 'party' in os.path.basename(target_file).lower():
        update_target_file(target_file, 'party_num', df_master_party)
    elif 'acct' in os.path.basename(target_file).lower():
        update_target_file(target_file, 'acct_num', df_master_account)


# Get user input for the directory containing the updated files
source_directory = browse_directory("Select the directory containing the updated files",default_output_directory)
# Get user input for the target directory to move the updated files
target_directory = browse_directory("Select the directory to move the updated files",default_updated_directory)
# Find all files with the prefix 'updated_' in the source directory
updated_files = [f for f in os.listdir(source_directory) if f.startswith('updated_')]

# Move each updated file to the target directory
for file_name in updated_files:
    source_path = os.path.join(source_directory, file_name)
    target_path = os.path.join(target_directory, file_name)
    shutil.move(source_path, target_path)
    print(f"Moved '{file_name}' to '{target_directory}'")

print("All updated files have been successfully moved!")

In [ ]:
#csv to json
import csv
import json
import glob
import os
from tkinter import Tk, filedialog

def browse_directory(prompt,default_path):
    Tk().withdraw()  # We don't want a full GUI, so keep the root window from appearing
    directory = filedialog.askdirectory(title=prompt,initialdir=default_path)
    return directory

# Set the default path for ease of use
default_csv_directory = os.path.join(os.getcwd(), "4.updated")
default_json_directory = os.path.join(os.getcwd(), "5.json")

# Get user input for the directory containing the CSV files
csv_directory = browse_directory("Select the directory containing the CSV files",default_csv_directory)

# Path to your CSV files
csv_files = glob.glob(os.path.join(csv_directory, '*.csv'))

# Get user input for the target directory to save JSON files
json_directory = browse_directory("Select the directory to save the JSON files",default_json_directory)

for csv_file in csv_files:
    # Read the CSV file
    with open(csv_file, mode='r', newline='') as file:
        reader = csv.DictReader(file)
        rows = list(reader)

    # Convert to JSON
    json_data = json.dumps(rows, indent=4)

    # Write to a JSON file in the selected directory
    json_file_name = os.path.splitext(os.path.basename(csv_file))[0].replace('updated_', '') + '.json'
    json_file_path = os.path.join(json_directory, json_file_name)
    with open(json_file_path, mode='w') as json_file:
        json_file.write(json_data)

    print(f"Converted {csv_file} to {json_file_path}")

print("All CSV files have been successfully converted to JSON!")

In [ ]:
## Insert Sql with json
import json
import os
import glob
from tkinter import Tk, filedialog

# Function to browse and select a directory
def browse_directory(prompt,default_path):
    Tk().withdraw()  # We don't want a full GUI, so keep the root window from appearing
    directory = filedialog.askdirectory(title=prompt,initialdir=default_path)
    return directory

# Set the default path for ease of use
default_json_directory = os.path.join(os.getcwd(), "5.json")
default_sql_directory = os.path.join(os.getcwd(), "6.sql")

# Get user input for the directory containing the JSON files
json_directory = browse_directory("Select the directory containing the JSON files",default_json_directory)

# Get user input for the target directory to save SQL files
output_directory = browse_directory("Select the directory to save the SQL files",default_sql_directory)

# Directory containing JSON files and the wildcard pattern
json_files_pattern = os.path.join(json_directory, '*.json')

# Find all JSON files matching the pattern
input_json_files = glob.glob(json_files_pattern)

# Data for non-JSON columns
BATCH_ID = '123456789'
LINE_OF_BUSINESS = 'MORTGAGE'
SNAPSHOT_DT = '12/31/2024'

# Iterate over each JSON file
for input_json_file in input_json_files:
    # Read JSON data from the input file
    with open(input_json_file, 'r') as file:
        json_data = json.load(file)

    table_name = os.path.splitext(os.path.basename(input_json_file))[0]
    # Output file path
    output_file_path = os.path.join(output_directory, f"{table_name}.sql")

    # Ensure the output directory exists
    os.makedirs(os.path.dirname(output_file_path), exist_ok=True)

    # Open the output file for writing
    with open(output_file_path, 'w') as output_file:
        # Iterate over each record in the JSON data
        for record in json_data:
            # Convert the current record to a JSON string
            json_str = json.dumps(record)

            # Generate the insert statement for the current record
            insert_sql = f"""
            INSERT INTO {table_name} (BATCH_ID, LINE_OF_BUSINESS, SNAPSHOT_DT, OBJECT_NAME, JSON_MESSAGE)
            VALUES ('{BATCH_ID}', '{LINE_OF_BUSINESS}', '{SNAPSHOT_DT}', '{json_str}');
            """

            # Write the insert statement to the output file
            output_file.write(insert_sql + "\n")

    print(f"Insert statements have been written to {output_file_path}")

print("All JSON files have been successfully converted to SQL insert statements!")

In [ ]:
#OLD WAY CONVERT TO JSON

import csv
import json
import os

def csv_to_json(csv_file_path, json_file_path):
    with open(csv_file_path, mode='r', encoding='utf-8') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        
        with open(json_file_path, mode='w', encoding='utf-8') as json_file:
            for row in csv_reader:
                json.dump(row, json_file)
                json_file.write('\n')

def convert_all_csv_to_json(directory):
    for filename in os.listdir(directory):
        if filename.endswith('.csv'):
            csv_file_path = os.path.join(directory, filename)
            json_file_path = os.path.join(directory, filename.replace('.csv', '.json'))
            csv_to_json(csv_file_path, json_file_path)
            print(f'Converted {csv_file_path} to {json_file_path}')

# Replace 'your_directory_path' with the path to your directory containing CSV files
convert_all_csv_to_json('output')



README
>>1.Generate Test Data Files based on schema
>>2.Generate Master Files Party, Account with record count greater than the mock data files
>>3.Replace Party , Account Number Details with Master Files to enforce Party, Account, Party Account are joined
>>4.Convert csv to json
>>5.Add non json fields + json field in a sql statement
